# @produces basics

In [ ]:
# | hide

from IPython.display import Markdown as md

from fastkafka.testing import LocalKafkaBroker, run_script_and_cancel
from fastkafka._testing.local_broker import run_and_match
from fastkafka._components._subprocess import terminate_asyncio_process

You can use `@produces` decorator to produce messages to Kafka topics. 

In this guide we will create a simple FastKafka app that will produce hello world messages to hello_world topic.

## Import `FastKafka`

To use the `@produces` decorator, frist we need to import the base FastKafka app to create our application.

In [ ]:
# | echo: false

import_fastkafka = """from fastkafka import FastKafka
"""

md(f"```python\n{import_fastkafka}\n```")

```python
from fastkafka import FastKafka

```

## Define the structure of the messages
Next, you need to define the structure of the messages you want to send to the topic using [pydantic](https://docs.pydantic.dev/). For the guide we'll stick to something basic, but you are free to define any complex message structure you wish in your project, just make sure it can be JSON encoded.

Let's import `BaseModel` and `Field` from pydantic and create a simple `HelloWorld` class containing one string parameter `msg`

In [ ]:
# | echo: false

import_pydantic = """from pydantic import BaseModel, Field
"""
md(f"```python\n{import_pydantic}\n```")

```python
from pydantic import BaseModel, Field

```

In [ ]:
# | echo: false

define_HelloWorld = """
class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )
"""
md(f"```python\n{define_HelloWorld}\n```")

```python

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

```

## Create a base FastKafka app

Now we will create and define a base FastKafka app, replace the \<address_of_your_kafka_bootstrap_server\> with the actual address of your Kafka bootstrap server

In [ ]:
# | echo: false

create_app = """app = FastKafka(bootstrap_servers="<address_of_your_kafka_bootstrap_server>")
"""
md(f"```python\n{create_app}\n```")

```python
app = FastKafka(bootstrap_servers="<address_of_your_kafka_bootstrap_server>")

```

## Create a producer function and decorate it with `@produces`

Let's create a producer function and explain its basic structure:

In [ ]:
# | echo: false

decorate_produces = """
@app.produces()
async def to_hello_world(msg: str) -> HelloWorld:
    return HelloWorld(msg=msg)
"""
md(f"```python\n{decorate_produces}\n```")

```python

@app.produces()
async def to_hello_world(msg: str) -> HelloWorld:
    return HelloWorld(msg=msg)

```

You can call your defined function as any normal python function in your code. The side effect of calling the function will be that the value you are returning will also be sent to a kafka topic.

## Instruct the app to start sending HelloWorld messages

Let's use `@run_in_background` decorator to instruct our app to send HelloWorld messages to hello_world topic every second.

In [ ]:
# | echo: false

define_run = """
import asyncio

@app.run_in_background()
async def hello_every_second():
    while(True):
        await to_hello_world(msg="Hello world!")
        await asyncio.sleep(1)
"""
md(f"```python\n{define_run}\n```")

```python

import asyncio

@app.run_in_background()
async def hello_every_second():
    while(True):
        await to_hello_world(msg="Hello world!")
        await asyncio.sleep(1)

```

## Final app

Your app code should look like this:

In [ ]:
# | echo: false

produces_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_produces
    + define_run
)
md(f"```python\n{produces_example}\n```")

```python
from fastkafka import FastKafka
from pydantic import BaseModel, Field

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )
app = FastKafka(bootstrap_servers="<address_of_your_kafka_bootstrap_server>")

@app.produces()
async def to_hello_world(msg: str) -> HelloWorld:
    return HelloWorld(msg=msg)

import asyncio

@app.run_in_background()
async def hello_every_second():
    while(True):
        await to_hello_world(msg="Hello world!")
        await asyncio.sleep(1)

```

## Run the app

In [ ]:
script_file = "producer_example.py"
cmd = "fastkafka run --num-workers=1 producer_example:app"
md(f"Now we can run the app. Copy the code above in producer_example.py and run it by running\n```shell\n{cmd}\n```")

Now we can run the app. Copy the code above in producer_example.py and run it by running
```shell
fastkafka run --num-workers=1 producer_example:app
```

After running the command, you should see this output in your terminal:

In [ ]:
# | echo: false

with LocalKafkaBroker(topicas=["hello_world"], apply_nest_asyncio=True) as bootstrap_server:
    exit_code, output = await run_script_and_cancel(
            script=produces_example.replace("<address_of_your_kafka_bootstrap_server>", bootstrap_server),
            script_file=script_file,
            cmd=cmd,
            cancel_after=5,
        )

    assert exit_code == 0

    print(output.decode("UTF-8"))

[130979]: [INFO] fastkafka._application.app: run_in_background() : Adding function 'hello_every_second' as background task
[130979]: [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9092'}'
[130979]: [INFO] fastkafka._application.app: _populate_bg_tasks() : Starting background task 'hello_every_second'
Starting process cleanup, this may take a few seconds...
[INFO] fastkafka._server: terminate_asyncio_process(): Terminating the process 130979...
[130979]: [INFO] fastkafka._application.app: _shutdown_bg_tasks() : Cancelling background task 'hello_every_second'
[130979]: [INFO] fastkafka._application.app: _shutdown_bg_tasks() : Waiting for background task 'hello_every_second' to finish
[130979]: [INFO] fastkafka._application.app: _shutdown_bg_tasks() : Execution finished for background task 'hello_every_second'
[INFO] fastkafka._server: terminate_asyncio_process(): Process 130979 terminated.



## Check if the message was sent to the Kafka topic

Lets check the topic and see if there is a "Hello world!" message in the hello_world topic. In your terminal run:

In [ ]:
# | echo: false

consumer_cmd = f"kafka-console-consumer.sh -topic=hello_world --from-beginning -bootstrap-server=<addr_of_your_kafka_bootstrap_server>"
md(f"```shell\n{consumer_cmd}\n```")

```shell
kafka-console-consumer.sh -topic=hello_world --from-beginning -bootstrap-server=<addr_of_your_kafka_bootstrap_server>
```

In [ ]:
# | hide


produces_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_produces
    + define_run
)

with LocalKafkaBroker(topics=["hello_world"], apply_nest_asyncio=True) as bootstrap_server:
    exit_code, output = await run_script_and_cancel(
            script=produces_example.replace("<address_of_your_kafka_bootstrap_server>", bootstrap_server),
            script_file=script_file,
            cmd=cmd,
            cancel_after=5,
        )

    assert exit_code == 0

    proc = await run_and_match(
        *consumer_cmd.replace(
            "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
        ).split(" "),
        pattern='{"msg": "Hello world!"}',
        timeout=10
    )
    
    await terminate_asyncio_process(proc)

You should see the {"msg": "Hello world!"} messages in your topic.

## Choosing a topic

You probably noticed that you didn't define which topic you are sending the message to, this is because the `@produces` decorator determines the topic by default from your function name.
The decorator will take your function name and strip the default "to_" prefix from it and use the rest as the topic name. In this example case, the topic is *hello_world*.

!!! warn \"New topics\"

    Kafka producers and application startup will fail if the topics you are producing to don't yet exist. Before running the app, make sure that the topics are created.

You can choose your custom prefix by defining the `prefix` parameter in produces decorator, like this:

In [ ]:
# | echo: False
decorate_produces_prefix = """
@app.produces(prefix="send_to_")
async def send_to_hello_world(msg: str) -> HelloWorld:
    return HelloWorld(msg=msg)
"""
md(f"```python\n{decorate_produces_prefix}\n```")

In [ ]:
# | hide


produces_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_produces_prefix
    + define_run.replace("to_hello_world", "send_to_hello_world")
)

with LocalKafkaBroker(topics=["hello_world"], apply_nest_asyncio=True) as bootstrap_server:
    exit_code, output = await run_script_and_cancel(
            script=produces_example.replace("<address_of_your_kafka_bootstrap_server>", bootstrap_server),
            script_file=script_file,
            cmd=cmd,
            cancel_after=5,
        )

    assert exit_code == 0

    proc = await run_and_match(
        *consumer_cmd.replace(
            "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
        ).split(" "),
        pattern='{"msg": "Hello world!"}',
        timeout=10
    )
    
    await terminate_asyncio_process(proc)

Also, you can define the topic name completely by defining the `topic` in parameter in produces decorator, like this:

In [ ]:
# | echo: False
decorate_produces_topic = """
@app.produces(topic="my_special_topic")
async def to_hello_world(msg: str) -> HelloWorld:
    return HelloWorld(msg=msg)
"""
md(f"```python\n{decorate_produces_topic}\n```")

In [ ]:
# | hide

produces_example = (
    import_fastkafka
    + import_pydantic
    + define_HelloWorld
    + create_app
    + decorate_produces_topic
    + define_run
)

with LocalKafkaBroker(topics=["my_special_topic"], apply_nest_asyncio=True) as bootstrap_server:
    exit_code, output = await run_script_and_cancel(
            script=produces_example.replace("<address_of_your_kafka_bootstrap_server>", bootstrap_server),
            script_file=script_file,
            cmd=cmd,
            cancel_after=5,
        )

    assert exit_code == 0

    proc = await run_and_match(
        *consumer_cmd.replace(
            "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
        ).replace("hello_world", "my_special_topic").split(" "),
        pattern='{"msg": "Hello world!"}',
        timeout=10
    )
    
    await terminate_asyncio_process(proc)

## Message data

What you return from your function will be translated to bytes and sent to defined Kafka topic. The typing of the return value is used for generating the documentation for your Kafka app.

In this example case, the return value is HelloWorld class which will be translated into JSON formatted string and then to bytes. The translated data will then be sent to Kafka.